In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

/Users/gmarchetti/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(100,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_shape=(self.latent_dim,)))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train.astype(np.float32) / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            # Sample noise and generate a half batch of new images
            noise = np.random.normal(0, 1, (half_batch, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Sample generator input
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, np.ones((batch_size, 1)))

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        #fig.suptitle("DCGAN: Generated digits", fontsize=12)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/mnist_%d.png" % epoch)
        plt.close()

In [3]:
if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=4000, batch_size=32, save_interval=50)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
lea

54 [D loss: 0.761615, acc.: 56.25%] [G loss: 1.230901]
55 [D loss: 0.887291, acc.: 59.38%] [G loss: 1.199026]
56 [D loss: 0.743576, acc.: 59.38%] [G loss: 1.090401]
57 [D loss: 0.879753, acc.: 43.75%] [G loss: 1.299518]
58 [D loss: 0.835530, acc.: 50.00%] [G loss: 1.396358]
59 [D loss: 1.017255, acc.: 46.88%] [G loss: 1.409983]
60 [D loss: 0.822716, acc.: 53.12%] [G loss: 1.314748]
61 [D loss: 0.689107, acc.: 59.38%] [G loss: 1.561886]
62 [D loss: 1.137642, acc.: 40.62%] [G loss: 0.897644]
63 [D loss: 1.010011, acc.: 40.62%] [G loss: 1.263265]
64 [D loss: 0.873451, acc.: 43.75%] [G loss: 1.501385]
65 [D loss: 1.152834, acc.: 34.38%] [G loss: 1.243223]
66 [D loss: 1.116136, acc.: 40.62%] [G loss: 0.799231]
67 [D loss: 0.856718, acc.: 43.75%] [G loss: 0.935010]
68 [D loss: 0.714520, acc.: 56.25%] [G loss: 1.258551]
69 [D loss: 0.969146, acc.: 40.62%] [G loss: 1.101008]
70 [D loss: 1.140781, acc.: 37.50%] [G loss: 1.164553]
71 [D loss: 0.816088, acc.: 56.25%] [G loss: 1.223045]
72 [D loss

202 [D loss: 0.814445, acc.: 40.62%] [G loss: 1.046814]
203 [D loss: 0.660147, acc.: 65.62%] [G loss: 1.227300]
204 [D loss: 0.767937, acc.: 53.12%] [G loss: 1.073393]
205 [D loss: 0.977737, acc.: 34.38%] [G loss: 1.292827]
206 [D loss: 0.981894, acc.: 40.62%] [G loss: 0.831053]
207 [D loss: 0.782379, acc.: 59.38%] [G loss: 1.036427]
208 [D loss: 0.932463, acc.: 40.62%] [G loss: 0.877389]
209 [D loss: 0.913394, acc.: 40.62%] [G loss: 0.955831]
210 [D loss: 0.682177, acc.: 59.38%] [G loss: 0.957643]
211 [D loss: 0.787136, acc.: 50.00%] [G loss: 1.100343]
212 [D loss: 0.833068, acc.: 46.88%] [G loss: 0.939904]
213 [D loss: 0.742692, acc.: 56.25%] [G loss: 0.915993]
214 [D loss: 0.866769, acc.: 43.75%] [G loss: 0.890773]
215 [D loss: 0.884784, acc.: 43.75%] [G loss: 0.831226]
216 [D loss: 0.817253, acc.: 53.12%] [G loss: 0.995447]
217 [D loss: 1.158293, acc.: 28.12%] [G loss: 0.907409]
218 [D loss: 0.851234, acc.: 56.25%] [G loss: 0.945735]
219 [D loss: 1.101068, acc.: 21.88%] [G loss: 1.

349 [D loss: 0.920253, acc.: 34.38%] [G loss: 0.934483]
350 [D loss: 1.018170, acc.: 43.75%] [G loss: 0.800584]
351 [D loss: 0.765761, acc.: 59.38%] [G loss: 0.935205]
352 [D loss: 0.704180, acc.: 56.25%] [G loss: 1.002105]
353 [D loss: 0.807234, acc.: 53.12%] [G loss: 0.812680]
354 [D loss: 0.821771, acc.: 46.88%] [G loss: 0.974165]
355 [D loss: 0.791894, acc.: 46.88%] [G loss: 1.100528]
356 [D loss: 0.794980, acc.: 62.50%] [G loss: 1.011261]
357 [D loss: 0.839580, acc.: 53.12%] [G loss: 0.896661]
358 [D loss: 0.595473, acc.: 68.75%] [G loss: 0.823685]
359 [D loss: 0.897785, acc.: 34.38%] [G loss: 0.852764]
360 [D loss: 0.800921, acc.: 46.88%] [G loss: 0.928401]
361 [D loss: 0.735107, acc.: 56.25%] [G loss: 0.954726]
362 [D loss: 0.763484, acc.: 59.38%] [G loss: 0.970211]
363 [D loss: 0.776160, acc.: 50.00%] [G loss: 1.167326]
364 [D loss: 0.815373, acc.: 40.62%] [G loss: 1.157499]
365 [D loss: 0.813697, acc.: 50.00%] [G loss: 1.136214]
366 [D loss: 0.897824, acc.: 43.75%] [G loss: 0.

496 [D loss: 0.772963, acc.: 53.12%] [G loss: 1.060486]
497 [D loss: 0.754366, acc.: 53.12%] [G loss: 1.090654]
498 [D loss: 0.756207, acc.: 53.12%] [G loss: 0.917194]
499 [D loss: 0.799490, acc.: 59.38%] [G loss: 0.901186]
500 [D loss: 0.842965, acc.: 50.00%] [G loss: 0.765044]
501 [D loss: 0.786575, acc.: 46.88%] [G loss: 0.792421]
502 [D loss: 0.873124, acc.: 37.50%] [G loss: 0.792170]
503 [D loss: 0.828071, acc.: 40.62%] [G loss: 1.010503]
504 [D loss: 0.743297, acc.: 62.50%] [G loss: 0.875188]
505 [D loss: 0.978482, acc.: 34.38%] [G loss: 0.909251]
506 [D loss: 0.708247, acc.: 56.25%] [G loss: 1.034342]
507 [D loss: 0.750449, acc.: 56.25%] [G loss: 0.956379]
508 [D loss: 0.883749, acc.: 46.88%] [G loss: 0.987502]
509 [D loss: 0.829214, acc.: 56.25%] [G loss: 0.791103]
510 [D loss: 0.832137, acc.: 46.88%] [G loss: 1.011256]
511 [D loss: 0.734605, acc.: 59.38%] [G loss: 1.056525]
512 [D loss: 0.859881, acc.: 43.75%] [G loss: 1.268271]
513 [D loss: 0.628632, acc.: 75.00%] [G loss: 1.

...

3386 [D loss: 0.754795, acc.: 43.75%] [G loss: 0.874041]
3387 [D loss: 0.644985, acc.: 71.88%] [G loss: 0.918619]
3388 [D loss: 0.716870, acc.: 53.12%] [G loss: 0.989196]
3389 [D loss: 0.561665, acc.: 65.62%] [G loss: 0.848849]
3390 [D loss: 0.672470, acc.: 65.62%] [G loss: 1.099248]
3391 [D loss: 0.553772, acc.: 71.88%] [G loss: 1.064360]
3392 [D loss: 0.723610, acc.: 56.25%] [G loss: 0.991929]
3393 [D loss: 0.635114, acc.: 56.25%] [G loss: 0.855444]
3394 [D loss: 0.784967, acc.: 40.62%] [G loss: 0.857764]
3395 [D loss: 0.643083, acc.: 71.88%] [G loss: 0.979003]
3396 [D loss: 0.567966, acc.: 84.38%] [G loss: 0.993777]
3397 [D loss: 0.622010, acc.: 65.62%] [G loss: 0.932416]
3398 [D loss: 0.803752, acc.: 50.00%] [G loss: 0.899963]
3399 [D loss: 0.612502, acc.: 71.88%] [G loss: 0.854803]
3400 [D loss: 0.654464, acc.: 59.38%] [G loss: 0.960789]
3401 [D loss: 0.690390, acc.: 50.00%] [G loss: 1.064105]
3402 [D loss: 0.686713, acc.: 56.25%] [G loss: 0.885528]
3403 [D loss: 0.616498, acc.: 6

3530 [D loss: 0.621597, acc.: 62.50%] [G loss: 0.881578]
3531 [D loss: 0.571598, acc.: 71.88%] [G loss: 0.837646]
3532 [D loss: 0.570926, acc.: 71.88%] [G loss: 0.975972]
3533 [D loss: 0.689050, acc.: 56.25%] [G loss: 0.832392]
3534 [D loss: 0.702717, acc.: 50.00%] [G loss: 0.842861]
3535 [D loss: 0.743771, acc.: 43.75%] [G loss: 0.891065]
3536 [D loss: 0.612001, acc.: 65.62%] [G loss: 0.920793]
3537 [D loss: 0.687880, acc.: 59.38%] [G loss: 0.912774]
3538 [D loss: 0.706996, acc.: 46.88%] [G loss: 0.953198]
3539 [D loss: 0.610765, acc.: 65.62%] [G loss: 0.797080]
3540 [D loss: 0.717772, acc.: 56.25%] [G loss: 0.654639]
3541 [D loss: 0.764915, acc.: 43.75%] [G loss: 0.841880]
3542 [D loss: 0.680022, acc.: 53.12%] [G loss: 0.803837]
3543 [D loss: 0.707627, acc.: 56.25%] [G loss: 0.856082]
3544 [D loss: 0.689228, acc.: 59.38%] [G loss: 0.808879]
3545 [D loss: 0.629201, acc.: 68.75%] [G loss: 0.789988]
3546 [D loss: 0.784228, acc.: 37.50%] [G loss: 0.745887]
3547 [D loss: 0.628503, acc.: 5

3674 [D loss: 0.701898, acc.: 46.88%] [G loss: 0.855904]
3675 [D loss: 0.689492, acc.: 56.25%] [G loss: 0.816741]
3676 [D loss: 0.618841, acc.: 68.75%] [G loss: 0.927045]
3677 [D loss: 0.662964, acc.: 56.25%] [G loss: 0.853550]
3678 [D loss: 0.620685, acc.: 68.75%] [G loss: 0.934139]
3679 [D loss: 0.652766, acc.: 56.25%] [G loss: 0.936422]
3680 [D loss: 0.765413, acc.: 53.12%] [G loss: 0.929401]
3681 [D loss: 0.708310, acc.: 59.38%] [G loss: 0.798277]
3682 [D loss: 0.598166, acc.: 65.62%] [G loss: 0.864695]
3683 [D loss: 0.635893, acc.: 68.75%] [G loss: 0.968288]
3684 [D loss: 0.594417, acc.: 68.75%] [G loss: 0.830010]
3685 [D loss: 0.707003, acc.: 53.12%] [G loss: 0.990613]
3686 [D loss: 0.700937, acc.: 53.12%] [G loss: 0.850480]
3687 [D loss: 0.717063, acc.: 53.12%] [G loss: 0.974321]
3688 [D loss: 0.711923, acc.: 50.00%] [G loss: 0.808623]
3689 [D loss: 0.758118, acc.: 43.75%] [G loss: 0.967745]
3690 [D loss: 0.744564, acc.: 46.88%] [G loss: 0.959548]
3691 [D loss: 0.795186, acc.: 3

3818 [D loss: 0.644543, acc.: 56.25%] [G loss: 0.733943]
3819 [D loss: 0.603473, acc.: 75.00%] [G loss: 0.897322]
3820 [D loss: 0.685559, acc.: 50.00%] [G loss: 0.806610]
3821 [D loss: 0.700028, acc.: 59.38%] [G loss: 0.708429]
3822 [D loss: 0.635507, acc.: 65.62%] [G loss: 0.828516]
3823 [D loss: 0.850953, acc.: 28.12%] [G loss: 0.815843]
3824 [D loss: 0.593097, acc.: 78.12%] [G loss: 0.928468]
3825 [D loss: 0.697226, acc.: 50.00%] [G loss: 0.976745]
3826 [D loss: 0.721917, acc.: 59.38%] [G loss: 0.793734]
3827 [D loss: 0.672750, acc.: 56.25%] [G loss: 1.017579]
3828 [D loss: 0.624994, acc.: 62.50%] [G loss: 0.857988]
3829 [D loss: 0.702234, acc.: 50.00%] [G loss: 0.942574]
3830 [D loss: 0.732059, acc.: 50.00%] [G loss: 0.922796]
3831 [D loss: 0.796232, acc.: 37.50%] [G loss: 0.870483]
3832 [D loss: 0.724977, acc.: 56.25%] [G loss: 0.851468]
3833 [D loss: 0.677835, acc.: 56.25%] [G loss: 0.913973]
3834 [D loss: 0.682561, acc.: 53.12%] [G loss: 0.870086]
3835 [D loss: 0.767681, acc.: 4

3962 [D loss: 0.657179, acc.: 62.50%] [G loss: 0.960063]
3963 [D loss: 0.752485, acc.: 46.88%] [G loss: 0.849945]
3964 [D loss: 0.659601, acc.: 62.50%] [G loss: 0.941860]
3965 [D loss: 0.653160, acc.: 62.50%] [G loss: 0.938302]
3966 [D loss: 0.847176, acc.: 37.50%] [G loss: 0.903481]
3967 [D loss: 0.661066, acc.: 56.25%] [G loss: 0.934733]
3968 [D loss: 0.725961, acc.: 50.00%] [G loss: 0.817440]
3969 [D loss: 0.596088, acc.: 65.62%] [G loss: 0.793464]
3970 [D loss: 0.597312, acc.: 65.62%] [G loss: 0.940192]
3971 [D loss: 0.708783, acc.: 62.50%] [G loss: 0.816293]
3972 [D loss: 0.920754, acc.: 28.12%] [G loss: 0.961998]
3973 [D loss: 0.711771, acc.: 62.50%] [G loss: 0.874859]
3974 [D loss: 0.749526, acc.: 50.00%] [G loss: 0.845575]
3975 [D loss: 0.489854, acc.: 78.12%] [G loss: 0.820836]
3976 [D loss: 0.862798, acc.: 34.38%] [G loss: 0.807445]
3977 [D loss: 0.673172, acc.: 53.12%] [G loss: 0.971579]
3978 [D loss: 0.672822, acc.: 59.38%] [G loss: 0.815343]
3979 [D loss: 0.702420, acc.: 5